***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [13]:
import numpy as np
import chart_studio
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data
from sklearn.linear_model import LinearRegression

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

# To remove date from index 
# reset_index(drop = False, inplace = True)

# To erase NaN values
# dropna()

<div class="alert alert-block alert-info">
<b>Load the historical data, transform in montly data and get percentage change</b>
</div>

In [2]:
# If working for any with a list like ['Stock name'....] know that the work flow is different
# R = pdr.DataReader(['FLRY3.SA'], 'yahoo', start = '2010/01/01', end = '2019/12/31')

R = data.DataReader('FLRY3.SA', 'yahoo', start = '2010/01/01', end = '2019/12/31')
R = R.resample('M').last().pct_change()
R.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28,-0.007444,-0.019497,-0.019939,0.025654,-0.897819,0.025654
2010-03-31,-0.011500,-0.005756,0.031299,-0.049525,37.067073,-0.049525
2010-04-30,-0.076884,-0.100000,-0.118867,-0.066842,-0.796092,-0.066842
2010-05-31,-0.014247,0.035088,0.032147,0.003948,-0.858602,0.003948


<div class="alert alert-block alert-info">
<b>Same as above but with expected return of the market with GSPC = S&P 500 </b>
</div>

In [3]:
Rm = data.DataReader('^GSPC', 'yahoo', start = '2010/01/01', end = '2019/12/31')
Rm = Rm.resample('M').last().pct_change()
Rm.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28,0.009841,0.024235,0.014242,0.028514,-0.271144,0.028514
2010-03-31,0.060800,0.062147,0.062234,0.058796,0.136660,0.058796
2010-04-30,0.028462,0.017628,0.029887,0.014759,0.348751,0.014759
2010-05-31,-0.087252,-0.085592,-0.086330,-0.081976,-0.194610,-0.081976


<div class="alert alert-block alert-info">
<b>Same as above but with risk-free investment with DGS1MO = One Month Treasury Constant Maturity Rate</b>
</div>

In [4]:
Rf = data.DataReader('DGS1MO', 'fred', start = '2010/02/01', end = '2019/12/31')
Rf = Rf.resample('M').last()
Rf = Rf/100
Rf.head()

,DGS1MO
DATE,
2010-02-28,0.0009
2010-03-31,0.0015
2010-04-30,0.0014
2010-05-31,0.0015
2010-06-30,0.0017


<div class="alert alert-block alert-info">
<b>Table for Capital Asset Pricing Model</b>
</div>

In [5]:
# When working with a list it's good to use concat method 
# CAPM = pd.concat([R['Adj Close'], Rm['Adj Close'], Rf['DGS1MO']], axis = 1)
# CAPM.rename(columns = {'FLRY3.SA': 'R', '^GSPC': 'Rm', 'DGS1MO': 'Rf'}, inplace = True)

CAPM = pd.DataFrame({'R': R['Adj Close'], 'Rm': Rm['Adj Close'], 'Rf': Rf['DGS1MO']}).dropna()
CAPM['R - Rf'] = CAPM['R'] - CAPM['Rf']
CAPM['Rm - Rf'] = CAPM['Rm'] - CAPM['Rf']
CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf
Date,,,,,
2010-02-28,0.025654,0.028514,0.0009,0.024754,0.027614
2010-03-31,-0.049525,0.058796,0.0015,-0.051025,0.057296
2010-04-30,-0.066842,0.014759,0.0014,-0.068242,0.013359
2010-05-31,0.003948,-0.081976,0.0015,0.002448,-0.083476
2010-06-30,0.123033,-0.053882,0.0017,0.121333,-0.055582


<div class="alert alert-block alert-info">
<b>Add deviation from the mean to the table</b>
</div>

In [6]:
mean = CAPM['R - Rf'].mean()
CAPM['Deviation from mean'] = CAPM['R - Rf'] - mean

CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf,Deviation from mean
Date,,,,,,
2010-02-28,0.025654,0.028514,0.0009,0.024754,0.027614,0.012437
2010-03-31,-0.049525,0.058796,0.0015,-0.051025,0.057296,-0.063342
2010-04-30,-0.066842,0.014759,0.0014,-0.068242,0.013359,-0.080560
2010-05-31,0.003948,-0.081976,0.0015,0.002448,-0.083476,-0.009869
2010-06-30,0.123033,-0.053882,0.0017,0.121333,-0.055582,0.109016


<div class="alert alert-block alert-info">
<b>Linear regression</b>
</div>

In [7]:
x = CAPM['Rm - Rf']
y = CAPM['R - Rf']

x = sm.add_constant(x)
model = sm.OLS(y, x, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 R - Rf   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     5.449
Date:                Mon, 04 May 2020   Prob (F-statistic):             0.0213
Time:                        17:35:44   Log-Likelihood:                 122.12
No. Observations:                 119   AIC:                            -240.2
Df Residuals:                     117   BIC:                            -234.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0100      0.008      1.242      0.217      -0.006       0.026
Rm - Rf        0.5104      0.219      2.334      0.021       0.077       0.943
==============================================================================
Omnibus:                        3.594   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.166   Jarque-Bera (JB):                3.005
Skew:                           0.353   Prob(JB):                        0.223
Kurtosis:                       3.327   Cond. No.                         27.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Add predicted values to table</b>
</div>

In [8]:
y_pred = results.predict()
CAPM['Predicted'] = y_pred
CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf,Deviation from mean,Predicted
Date,,,,,,,
2010-02-28,0.025654,0.028514,0.0009,0.024754,0.027614,0.012437,0.024126
2010-03-31,-0.049525,0.058796,0.0015,-0.051025,0.057296,-0.063342,0.039275
2010-04-30,-0.066842,0.014759,0.0014,-0.068242,0.013359,-0.080560,0.016851
2010-05-31,0.003948,-0.081976,0.0015,0.002448,-0.083476,-0.009869,-0.032572
2010-06-30,0.123033,-0.053882,0.0017,0.121333,-0.055582,0.109016,-0.018335


<div class="alert alert-block alert-info">
<b>Add residuals to table</b>
</div>

In [9]:
CAPM['Residual'] = CAPM['R - Rf'] - CAPM['Predicted']
CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf,Deviation from mean,Predicted,Residual
Date,,,,,,,,
2010-02-28,0.025654,0.028514,0.0009,0.024754,0.027614,0.012437,0.024126,0.000628
2010-03-31,-0.049525,0.058796,0.0015,-0.051025,0.057296,-0.063342,0.039275,-0.090300
2010-04-30,-0.066842,0.014759,0.0014,-0.068242,0.013359,-0.080560,0.016851,-0.085093
2010-05-31,0.003948,-0.081976,0.0015,0.002448,-0.083476,-0.009869,-0.032572,0.035020
2010-06-30,0.123033,-0.053882,0.0017,0.121333,-0.055582,0.109016,-0.018335,0.139669


<div class="alert alert-block alert-info">
<b>Linear regression plot</b>
</div>

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = CAPM['R - Rf'], 
    name = 'Actual', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 255, 145)')))

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = y_pred, 
    name = 'Predicted', 
    mode = 'lines',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(145, 255, 255)')))

fig.update_layout(title_text = 'CAPM - FLRY3 and S&P 500', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(x = 0, y = 1.5),
    xaxis_title = 'Beta',
    yaxis_title = 'Monthly expected return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel= dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Capital Asset Pricing Model using FLRY3 and S&P 500', auto_open = False)

$$Y_i = b_0 + b_1 X_i + \epsilon_i$$ <br/>
$$(R\ -\ Rf)\ =\ b_{0}\ +\ b_{1}(Rm\ -\ Rf)\ +\ \epsilon$$

<div class="alert alert-block alert-info">
<b>Add standard error of estimate to table</b>
</div>

In [11]:
x_pred = np.linspace(CAPM['Rm - Rf'].min(), CAPM['Rm - Rf'].max(), 119)
x_pred2 = sm.add_constant(x_pred)
y_pred2 = results.predict(x_pred2)

CAPM['SEE'] = y_pred2
CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf,Deviation from mean,Predicted,Residual,SEE
Date,,,,,,,,,
2010-02-28,0.025654,0.028514,0.0009,0.024754,0.027614,0.012437,0.024126,0.000628,-0.049262
2010-03-31,-0.049525,0.058796,0.0015,-0.051025,0.057296,-0.063342,0.039275,-0.090300,-0.048294
2010-04-30,-0.066842,0.014759,0.0014,-0.068242,0.013359,-0.080560,0.016851,-0.085093,-0.047326
2010-05-31,0.003948,-0.081976,0.0015,0.002448,-0.083476,-0.009869,-0.032572,0.035020,-0.046359
2010-06-30,0.123033,-0.053882,0.0017,0.121333,-0.055582,0.109016,-0.018335,0.139669,-0.045391


<div class="alert alert-block alert-info">
<b>Standard error of estimate plot</b>
</div>

In [12]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = CAPM['R - Rf'], 
    name = 'Actual', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 255, 145)')))

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = y_pred2, 
    name = 'Standard error of estimate', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 145, 255)')))

fig.update_layout(title_text = 'SEE of CAPM - FLRY3 and S&P 500', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(x = 0, y = 1.5),
    xaxis_title = 'Beta',
    yaxis_title = 'Monthly expected return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel= dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Standard Error of Estimate of CAPM using FLRY3 and S&P 500', auto_open = False)

$$SEE = \Bigg( \frac{\sum_{i=1}^n (Y_i - \hat{b}_0-\hat{b}_1X_i)^2}{n-2} \Bigg)^\frac{1}{2} = \Bigg( \frac{\sum_{i=1}^n (\hat\epsilon_i)^2}{n-2} \Bigg)^\frac{1}{2}$$